In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import html5lib
import selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException,ElementNotVisibleException
import re
import time
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

### 1 and 2. Write a python program which searches all the product under a particular product vertical from www.amazon.in. The product verticals to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product vertical has less than 3 pages in search results then scrape all the products available under that product vertical. Details to be scraped are: "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [37]:
brand_name = []
name_of_the_product = []
rating = []
no_of_ratings = []
price = []
return_exchange = []
expected_delivery = []
availability = []
other_details = []
product_URL = []

In [39]:
def amazon_product(url):
    
    product = input('Enter product type :')
    driver = webdriver.Chrome(r'C:\Users\Rahul\Downloads\chromedriver_win32\chromedriver.exe',chrome_options=options)
    driver.get(url)
    
    search = driver.find_element_by_xpath("//input[@id='twotabsearchtextbox']")
    search.send_keys(product)
    
    search_click = driver.find_element_by_xpath("//input[@id='nav-search-submit-button']")
    search_click.click()
    

    
    for page in range(0,3):
#getting product href   
        
        prd_link = driver.find_elements_by_xpath("//a[@class='a-link-normal a-text-normal']")
        for i in prd_link:
            product_URL.append(i.get_attribute('href'))
            
        next_button = driver.find_element_by_xpath("//li[@class='a-last']/a")
        if next_button.text == 'Next→':
            driver.get(next_button.get_attribute('href'))
        else:
            break
   
    
    for i in product_URL:
        driver.get(i)
        
        #product title and brand name
        title = driver.find_element_by_xpath("//span[@id='productTitle']")
        name_of_the_product.append(title.text)
        
        #number of ratings
        try:
            rat_tag = driver.find_element_by_xpath("//span[@id='acrCustomerReviewText']")
            no_of_ratings.append(rat_tag.text)
        except NoSuchElementException as e:
            no_of_ratings.append('-')
            
            
        #ratings star
        try:
            rat_tag = driver.find_element_by_xpath("//div[@id='averageCustomerReviews']")
            rat_tag.click() #this should be done as selenium rating option is not visible and selenium needs visible data to scrape
            rat_n = driver.find_element_by_xpath("//span[@class='a-size-medium a-color-base']")
            rating.append(rat_n.text)
        
        except NoSuchElementException as e:
            rating.append('-')

        
        #price
        try:
            prc_tag = driver.find_element_by_xpath("//td[@class='a-span12']/span")
            price.append(prc_tag.text)
        except NoSuchElementException as e:
            price.append('-')

        
        #exchange/return
        try:
            ex_rt = driver.find_element_by_xpath("//div[@id='RETURNS_POLICY']")
            return_exchange.append(ex_rt.text)
        except NoSuchElementException as e:
            return_exchange.append('-')

            
        #delivery
        try:
            del_tag = driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']/b")
            expected_delivery.append(del_tag.text)
        except NoSuchElementException as e:
            expected_delivery.append('-')


        #availability
        try:
            avl_tag = driver.find_element_by_xpath("//div[@id='availability']")
            availability.append(avl_tag.text)
        except NoSuchElementException as e:
            availability.append('-')


        #brand_name
        try:
            brand = driver.find_element_by_xpath("//a[@id='bylineInfo']")
            brand_name.append(brand.text)
        except NoSuchElementException as e:
            brand_name.append('-')
        
        
        #other details
        try:                                                                                    
            dtls = driver.find_element_by_xpath('//ul[@class="a-unordered-list a-vertical a-spacing-mini"]')
            other_details.append('  ||  '.join(dtls.text.split('\n')))
        except NoSuchElementException:
            other_details.append('-')
                
#     print(len(product_URL))


In [40]:
amazon_product('https://www.amazon.in/')

Enter product type :Mobile


In [41]:
print(len(brand_name))
print(len(name_of_the_product))
print(len(rating))
print(len(no_of_ratings))
print(len(price))
print(len(return_exchange))
print(len(expected_delivery))
print(len(availability))
print(len(other_details))
print(len(product_URL))

66
66
66
66
66
66
66
66
66
66


In [42]:
sol2_data = pd.DataFrame({
    'brand':brand_name,
    'name':name_of_the_product,
    'rating':rating,
    'number of ratings':no_of_ratings,
    'price':price,
    'return/exchange':return_exchange,
    'delivery':expected_delivery,
    'availability':availability,
    'other details': other_details
})


sol2_data.to_csv('sol2_data.csv')

sol2_data

,brand,name,rating,number of ratings,price,return/exchange,delivery,availability,other details
0,Visit the Samsung Store,"Samsung Galaxy M51 (Electric Blue, 6GB RAM, 12...",4.4 out of 5,"30,580 ratings","₹ 22,999.00",7 Days Replacement,"Saturday, Feb 27",In stock.,Quad camera setup - 64MP (F1.8) main camera + ...
1,Visit the Apple Store,New Apple iPhone 12 Mini (128GB) - Black,4.3 out of 5,539 ratings,"₹ 69,900.00",-,"Friday, Feb 26",In stock.,5.4-inch (13.7 cm diagonal) Super Retina XDR d...
2,Brand: Redmi,"Redmi 9A (Nature Green, 2GB Ram, 32GB Storage)...",4.1 out of 5,"13,689 ratings","₹ 6,799.00",7 Days Replacement,"Saturday, Feb 27",In stock.,"Country Of Origin - India, China || 13MP rea..."
3,Brand: Redmi,"Redmi 9 (Sky Blue, 4GB RAM, 64GB Storage)",4 out of 5,"11,194 ratings","₹ 8,799.00",7 Days Replacement,"Friday, Feb 26",In stock.,"Country Of Origin - India, China || 13+2MP R..."
4,Brand: Redmi,"Redmi 9A (Midnight Black, 3GB Ram, 32GB Storag...",4.1 out of 5,"13,689 ratings","₹ 7,499.00",7 Days Replacement,"Monday, March 1",In stock.,"13MP rear camera with AI portrait, AI scene re..."
...,...,...,...,...,...,...,...,...,...
61,Visit the Oppo Store,"Oppo A52 (Twilight Black, 6GB RAM, 128GB Stora...",4.1 out of 5,"4,217 ratings","₹ 14,990.00",7 Days Replacement,"Friday, Feb 26",In stock.,"12MP+8MP+2MP+2MP AI quad rear camera, 4K video..."
62,Brand: Redmi,"Redmi Note 9 Pro Max (Aurora Blue, 6GB RAM, 64...",4.1 out of 5,"30,057 ratings","₹ 14,999.00",7 Days Replacement,"Friday, Sep 10","In stock on September 8, 2021.\nOrder it now.",Country Of Origin - India || 64MP rear camer...
63,Visit the Oppo Store,"OPPO A31 (Lake Green, 4GB RAM, 64GB Storage) w...",4 out of 5,"2,939 ratings","₹ 9,990.00",7 Days Replacement,Feb 26 - March 3,"In stock on February 26, 2021.\nOrder it now.",12+2+2MP triple rear camera with portrait boke...
64,Visit the IKALL Store,I KALL K280 4G Smartphone (6.26 Inch IPS Displ...,4 out of 5,1 rating,"₹ 5,599.00",7 Days Replacement,"Saturday, Feb 27",In stock.,6.26 inch water drop Display | Made in India |...


In [43]:
brand_name.clear()
name_of_the_product.clear()
rating.clear()
no_of_ratings.clear()
price.clear()
return_exchange.clear()
expected_delivery.clear()
availability.clear()
other_details.clear()
product_URL.clear()

## 7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in. 

In [1]:
laptop_brand = []
laptop_model = []
os = []
display = []
processor = []
memory = []
weight = []
dimension = []
gpu = []
price = []
spec_link = []
seller = []

def digit_in(url):
    driver = webdriver.Chrome(r'C:\Users\Rahul\Downloads\chromedriver_win32\chromedriver.exe')
    driver.get(url)
    
    #clicking top 10
    top10 = driver.find_element_by_xpath("//div[@class='menu']/ul/li[3]")
    top10.click()
    
    time.sleep(1)
    
    #clicking best gaming laptops
    game = driver.find_element_by_xpath("//div[@class='listing_container']/ul/li[26]")
    game.click()
    
    #lisiting laptop's name
    brand_name = driver.find_elements_by_xpath("//div[@class='TopNumbeHeading active sticky-footer']/a")
    for i in brand_name:
        laptop_brand.append(i.text.split(' ')[0])
        laptop_model.append(i.text.split(" ", 1)[1])

#section centre specs
#os
    os_tag = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[3]")
    for i in os_tag:
        os.append(i.text.split(':')[1])
#display  
    disp_tag = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[4]")
    for i in disp_tag:
        display.append(i.text.split(':')[1])
#Processor        
    proc_tag = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[5]")
    for i in proc_tag:
        processor.append(i.text.split(':')[1])
#Memory
    mem_tag = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[6]")
    for i in mem_tag:
        memory.append(i.text.split(':')[1])
#Weight
    wght_tag = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[7]")
    for i in wght_tag:
        weight.append(i.text.split(':')[1])
#Dimension
    dim_tag = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[8]")
    for i in dim_tag:
        dimension.append(i.text.split(':')[1])
#Graphics Processor
    gpu_tag = driver.find_elements_by_xpath("//div[@class='Spcs-details']/table/tbody/tr[9]")
    for i in gpu_tag:
        gpu.append(i.text.split(':')[1])

# #price
    sell_tag = driver.find_elements_by_xpath("//table[@id='summtable']/tbody/tr")
    for i in sell_tag[1:11]:
        price.append(i.text.split(' ')[-1])
        seller.append(i.text.split(' ')[-2])
        

In [5]:
digit_in('https://www.digit.in/')

In [6]:
sol7_data = pd.DataFrame({
    'brand': laptop_brand,
    'model' : laptop_model,
    'os': os,
    'display': display,
    'processor': processor,
    'memory': memory,
    'weight': weight,
    'dimension': dimension,
    'gpu': gpu,
    'price': price,
    'seller': seller
})

In [7]:
sol7_data

,brand,model,os,display,processor,memory,weight,dimension,gpu,price,seller
0,MSI,GT76 TITAN DT 9SG,Windows 10 Pro,"17.3"" (UHD 3840x2160)",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB HDD/64 GBGB DDR4,4.2 kg,397 x 330 x 33~42 mm,NVIDIA GeForce RTX 2080,₹379990,N/A
1,ALIENWARE,17 AREA-51M,Windows 10 Pro,"17.3"" (FHD (1920 x 1080))",Intel 9th Gen Core i9-9900K | 5000 MHz,1 TB PCIe SSD/32GB DDR4,3.87 Kg,42 mm x 402.6 mm x 319.14 mm,NVIDIA GeForce RTX 2080,₹489909,amazon
2,HP,OMEN 15 2020,Windows 10 Home,"15.6"" (1920 x 1080)",Intel i7-10750H 10th Gen | 1.6GHz,512 GB SSD/16 GBGB DDR4,5.40,14.09 x 9.44 x 0.89,Nvidia GeForce GTX 1650Ti,₹117790,Tatacliq
3,ASUS,ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,₹164990,amazon
4,LENOVO,LEGION Y540,Windows 10 Home,"15.6"" (1920 X 1080)",9th Generation Core Intel I7-9750H | 2.6 Ghz,1 TB SSD/8GB DDR4,2.3,365mm x 260mm x 25.9mm,NVIDIA® GeForce RTX™ 2060,₹79990,flipkart
5,ASUS,ROG ZEPHYRUS G GA502,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen 7 Quad Core 3750H | 2.3 GHz,512 GB SSD/16GB DDR4,2.2,360 x 252 x 20.4,NVIDIA Geforce GTX 1660 Ti,₹79990,flipkart
6,ASUS,ROG ZEPHYRUS S GX531,Windows 10 Home,"15.6"" (1920 X 1080)",9th Gen Intel Core i7-8750H | 2.2 GHz,512GB SSD/16 GBGB DDR4,2.1,360 (W) x 268 (D) x 15.35~16.15 (H) mm,NVIDIA® GeForce RTX™ 2070 (Max-Q),₹146000,amazon
7,MSI,GT83VR 7RE TITAN SLI,Windows 10 Home 64 bit,"18.4"" (1920 x 1080)",Intel CM238 Core i7-7820HK+CM238 7th Gen | 3....,1.5 TB SATA/64GB DDR4,5.5,458 x 339 x 69,Dual GTX1070,₹349990,N/A
8,ASUS,ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,₹268990,amazon
9,DELL,G3,Windows 10 Home Plus,15.6 MP | NA,8th Generation Intel® Core™ i7-8750H | NA,256 + 1 TB SSD + SATA/16GB DDR4,2.53,22.7 X 380 X 258,NVIDIA® GeForce® GTX,₹61990,amazon


In [8]:
sol7_data.to_csv('sol7_data.csv')

## 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [15]:
driver = webdriver.Chrome(r"C:\Users\Rahul\Downloads\chromedriver_win32\chromedriver.exe")
driver.get('https://trak.in/')
driver.maximize_window()

In [16]:
#clicking on funding deal
driver.find_element_by_xpath("//li[@id='menu-item-51510']").click()

In [17]:
date = []
startup = []
industry = []
sub_vertical = []
city = []
investor = []
investment_type = []
amount = []

In [26]:
#the 3rd wuarter details are in table 48,49 and 50 so I will apply a loop for a range of 49-51 and clicking on show
#100 values in option button so that we will get all the details on each table

for i in range(48,51):
    driver.find_element_by_xpath("//div[@id='tablepress-{}_wrapper']/div/label/select/option[4]".format(i)).click()
    
#     #extracting date data
#     date_tag = driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[2]".format(i))
#     for i in date_tag:
#         date.append(i.text)

#     #extracting startup name
#     startup_name_tag = driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[3]".format(i))
#     for i in startup_name_tag:
#         startup.append(i.text)
        
#     #extracting Industry tag
#     ind_tag = driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[4]".format(i))
#     for i in ind_tag:
#         industry.append(i.text)

#     #sub_vertical
#     sub_v_tag = driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[5]".format(i))
#     for i in sub_v_tag:
#         sub_vertical.append(i.text)

#     #city
#     city_tag = driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[6]".format(i))
#     for i in city_tag:
#         city.append(i.text)

#     #Investor
#     investor_tag = driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[7]".format(i))
#     for i in investor_tag:
#         investor.append(i.text)

#     #Investment type
#     inv_type = driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[8]".format(i))
#     for i in inv_type:
#         investment_type.append(i.text)

#     #amount of investment
#     amount_tag = driver.find_elements_by_xpath("//table[@id='tablepress-{}']/tbody/tr/td[9]".format(i))
#     for i in amount_tag:
#         amount.append(i.text)

In [27]:
sol6_data = pd.DataFrame({
    'date': date,
    'startup': startup,
    'industry':industry,
    'sub vertical': sub_vertical,
    'city': city,
    'investor': investor,
    'investment type': investment_type,
    'amount': amount
})

In [28]:
sol6_data

,date,startup,industry,sub vertical,city,investor,investment type,amount
0,15/07/2020,Flipkart,E-commerce,E-commerce,Bangalore,Walmart Inc,M&A,"1,200,000,000"
1,16/07/2020,Vedantu,EduTech,Online Tutoring,Bangalore,Coatue Management,Series D,"100,000,000"
2,16/07/2020,Crio,EduTech,Learning Platform for Developers,Bangalore,021 Capital,pre-Series A,"934,160"
3,14/07/2020,goDutch,FinTech,Group Payments,Mumbai,"Matrix India,Y Combinator, Global Founders Cap...",Seed,"1,700,000"
4,13/07/2020,Mystifly,Airfare Marketplace,"Ticketing, Airline Retailing, and Post-Ticketi...",Singapore and Bangalore,Recruit Co. Ltd.,pre-Series B,"3,300,000"
5,09/07/2020,JetSynthesys,Gaming and Entertainment,Gaming and Entertainment,Pune,Adar Poonawalla and Kris Gopalakrishnan.,Venture-Series Unknown,"400,000"
6,10/07/2020,gigIndia,Marketplace,"Crowd Sourcing, Freelance",Pune,Incubate Fund India and Beyond Next Ventures,pre-Series A,"974,200"
7,15/07/2020,PumPumPum,Automotive Rental,Used Car-leasing platform,Gurgaon,Early Adapters Syndicate,Seed,"292,800"
8,14/07/2020,FLYX,OTT Player,Streaming Social Network,New York and Delhi,"Raj Mishra, founder of AIT Global Inc",pre-Seed,"200,000"
9,13/07/2020,Open Appliances Pvt. Ltd.,Information Technology,Internet-of-Things Security Solutions,Bangalore,Unicorn India Ventures,Venture-Series Unknown,"500,000"


In [29]:
sol6_data.to_csv('sol6_data.csv')

## 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’.

In [47]:
image_url = []

In [48]:
def google_images(url):
    
    image = input('Enter the name : ')
    driver = webdriver.Chrome(r'C:\Users\Rahul\Downloads\chromedriver_win32\chromedriver.exe')
    driver.get(url)
    driver.maximize_window() #maximizing browser windows
    image_tag = driver.find_element_by_xpath("//div[@class='gb_h gb_i'][2]")
    image_tag.click()
    #serach bar
    search_bar = driver.find_element_by_xpath("//input[@class='gLFyf gsfi']")
    search_bar.send_keys(image)
    #searchbutton
    search_butt = driver.find_element_by_xpath("//button[@class='Tg7LZd']")
    search_butt.click()
    
    #scrolling till the end of page
    for i in range(300):
        driver.execute_script("window.scrollBy(0,document.body.scrollHeight)")
    #for help check below link    
    #https://stackoverflow.com/questions/32391303/how-to-scroll-to-the-end-of-the-page-using-selenium-in-python
        
    #getting image elements
    img = driver.find_elements_by_xpath("//img[@class='rg_i Q4LuWd']")
#     print(len(img))
    for i in img:
        img_link = i.get_attribute('src') #getting image link
        if img_link is not None:   #checking only element where there is a link present
            if(img_link[0:5]=='https'): #checking 1st 5 letter of link is https or not
                image_url.append(img_link)
#     print(len(image_url))

    for i in range(len(image_url)):
        if i >= 100:
            break
        print("Downloading", i , "of", 100)
        response= requests.get(image_url[i])
        file = open("C:/Flip Robo/Cars/img"+str(i)+".jpg", "wb") #saaving files in folder in c drive
        file.write(response.content)
        

In [49]:
google_images("https://www.google.com/")

Enter the name : Cars


## 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.


In [50]:
product_url = []
brand_name = []
model_name = []
color = []
ram = []
rom = []
processor = []
no_processor = []
battery = []
price = []
prim_cam = []
sec_cam = []
disp_size = []
disp_res = []

In [51]:
def flipkart_products(url):
    
    #setting input
    product = input('Enter your product here :')
    
    
    #loading driver
    driver = webdriver.Chrome(r'C:\Users\Rahul\Downloads\chromedriver_win32\chromedriver.exe')
    driver.get(url)
    #clicking cross button
    driver.find_element_by_class_name('_2doB4z').click()
    
    #search bar and clicking on search button
    search_bar = driver.find_element_by_xpath("//div[@class='_3OO5Xc']/input")
    search_bar.send_keys(product)
    search_button = driver.find_element_by_xpath("//button[@class='L0Z3Pu']")
    search_button.click()
    

    #getting products url
    WebDriverWait(driver,2).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1fQZEK')))
    url = driver.find_elements_by_xpath("//a[@class='_1fQZEK']")
#     print(len(url))
    for i in url:
        product_url.append(i.get_attribute('href'))
        
#     getting each product url and then scraping required data
    for i in product_url[0:39]:
        driver.get(i)
        #scraping brand names:
        try:
            brand_tag = driver.find_element_by_xpath("//span[@class='B_NuCI']")
            brand_name.append(brand_tag.text.split(' ')[0])
        except NoSuchElementException:
            print('-')
        
        
#         scraping smartphone name
        try:
            phone_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][1]/table/tbody/tr[3]/td[2]//ul/li")
            model_name.append(phone_tag.text)
        except NoSuchElementException as e:
            model_name.append('-')

        #color of smartphone
        try:
            col_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][1]/table/tbody/tr[4]/td[2]//ul/li")
            color.append(col_tag.text)
        except NoSuchElementException as e:
            color.append('-')
            
            
        #clicking on readmore button
        try:
            WebDriverWait(driver,1).until(EC.presence_of_all_elements_located((By.XPATH,"//button[@class='_2KpZ6l _1FH0tX']")))
            read_more = driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']")
            read_more.click()
        except NoSuchElementException as e:
            print("can't move to next page")
            
            
        #Ram and Rom
        try:
            ram_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][4]/div")
            if ram_tag.text != "Memory & Storage Features" : raise NoSuchElementException
            ram_tag1 = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][4]/table/tbody/tr[2]/td[2]/ul/li")
            ram.append(ram_tag1.text)
        except NoSuchElementException as e:
            ram.append('-')

         
        try:
            ram_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][4]/div")
            if ram_tag.text != "Memory & Storage Features" : raise NoSuchElementException
            rom_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][4]/table/tbody/tr[1]/td[2]/ul/li")
            rom.append(rom_tag.text)
        except NoSuchElementException as e:
            rom.append('-')


        #primary camera
        try:
            prm_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][5]/div")
            if prm_tag.text != 'Camera Features': raise NoSuchElementException
            prm_tag1 = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][5]/table/tbody/tr[2]/td[2]/ul/li")
            prim_cam.append(prm_tag1.text)
        except NoSuchElementException as e:
            prim_cam.append('-')


        #secondary camera
        try:
            cam_tag_check = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][5]/table/tbody/tr[5]/td[1]")
            cam_tag_check2 = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][5]/table/tbody/tr[6]/td[1]")
            if cam_tag_check.text =='Secondary Camera':
                sec_cam.append(driver.find_element_by_xpath("//div[@class='_3k-BhJ'][5]/table/tbody/tr[5]/td[2]/ul/li").text)
            elif cam_tag_check2.text == 'Secondary Camera':
                sec_cam.append(driver.find_element_by_xpath("//div[@class='_3k-BhJ'][5]/table/tbody/tr[6]/td[2]/ul/li").text)
            else:
                sec_cam.append('-')
        except NoSuchElementException:
            sec_cam.append('-')


        #display size and resolution
        try:
            disp_size_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][2]/div")
            if disp_size_tag.text != 'Display Features' : raise NoSuchElementException
            disp_size_data = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][2]/table/tbody/tr[1]/td[2]/ul/li")
            disp_size.append(disp_size_data.text)
        except NoSuchElementException:
            disp_size.append('-')


        #display size and resolution
        try:
            disp_res_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][2]/div")
            if disp_res_tag.text != 'Display Features' : raise NoSuchElementException
            disp_res_data = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][2]/table/tbody/tr[2]/td[2]/ul/li")
            disp_res.append(disp_res_data.text)
        except NoSuchElementException:
            disp_res.append('-')

        
        #processor
        try:
            proc_tag = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/div")
            if proc_tag.text != "Os & Processor Features": raise NoSuchElementException
            proc_name = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/table/tbody/tr[2]/td[2]/ul/li")
            processor.append(proc_name.text)
        except NoSuchElementException:
            processor.append('-')


        #number of processor cores
        try:
            core_chek = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/div")
            if core_chek.text != "Os & Processor Features": raise NoSuchElementException
            core_1 = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/table/tbody/tr[2]/td[1]")
            core_2 = driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/table/tbody/tr[3]/td[1]")
            if core_1.text == "Processor Core":
                no_processor.append(driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/table/tbody/tr[2]/td[2]/ul/li").text)
            elif core_2.text == "Processor Core":
                no_processor.append(driver.find_element_by_xpath("//div[@class='_3k-BhJ'][3]/table/tbody/tr[3]/td[2]/ul/li").text)
            else:
                no_processor.append('-')
        except NoSuchElementException:
            no_processor.append('-')



        #battery capacity
        try:
            if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
                if driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                    bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                    if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                    bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
                elif driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                    bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                    if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                    bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
                else:
                    raise NoSuchElementException
            else :
                bat_chk = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
                if bat_chk.text != "Battery Capacity" : raise NoSuchElementException
                bat_cap = driver.find_element_by_xpath('//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')                # Extracting Availability from xpath
            battery.append(bat_cap.text)
        except NoSuchElementException:
            battery.append('-')
    
    
    
        #price of the product
        try:
            prc_tag = driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']")
            price.append(prc_tag.text)
        except NoSuchElementException:
            price.append('-')

In [52]:
flipkart_products('https://www.flipkart.com/')

Enter your product here :Galaxy


In [53]:
print(len(product_url))
print(len(brand_name))
print(len(model_name))
print(len(color))
print(len(ram))
print(len(rom))
print(len(processor))
print(len(no_processor))
print(len(battery))
print(len(price)) 
print(len(prim_cam))
print(len(sec_cam))
print(len(disp_size))
print(len(disp_res))

40
39
39
39
39
39
39
39
39
39
39
39
39
39


In [54]:
sol4_data = pd.DataFrame({
    'brand':brand_name,
    'model':model_name,
    'color':color,
    'ram':ram,
    'storage':rom,
    'processor':processor,
    'num_processor':no_processor,
    'batter_capacity':battery,
    'primar_camera': prim_cam,
    'secondar_camera':sec_cam,
    'display_size':disp_size,
    'display_resolution':disp_res,
    'price':price,
    'product_url': product_url[0:39]
})

sol4_data.to_csv('sol4_data.csv')

sol4_data

,brand,model,color,ram,storage,processor,num_processor,batter_capacity,primar_camera,secondar_camera,display_size,display_resolution,price,product_url
0,Samsung,Galaxy F41,Fusion Blue,6 GB,64 GB,Exynos 9611,Octa Core,6000 mAh,64MP + 8MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),2340 x 1080 Pixels,"₹15,499",https://www.flipkart.com/samsung-galaxy-f41-fu...
1,Samsung,Galaxy F62,Laser Green,6 GB,128 GB,Exynos 9825,Octa Core,7000 mAh,64MP + 12MP + 5MP + 5MP,32MP Front Camera,17.02 cm (6.7 inch),2400 x 1080 Pixels,"₹23,999",https://www.flipkart.com/samsung-galaxy-f62-la...
2,Samsung,Galaxy F41,Fusion Black,6 GB,64 GB,Exynos 9611,Octa Core,6000 mAh,64MP + 8MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),2340 x 1080 Pixels,"₹15,499",https://www.flipkart.com/samsung-galaxy-f41-fu...
3,LuvLap,-,-,-,-,-,-,-,-,-,-,-,"₹1,349",https://www.flipkart.com/luvlap-galaxy-baby-ca...
4,Samsung,Galaxy F41,Fusion Green,6 GB,64 GB,Exynos 9611,Octa Core,6000 mAh,64MP + 8MP + 5MP,32MP Front Camera,16.26 cm (6.4 inch),2340 x 1080 Pixels,"₹15,499",https://www.flipkart.com/samsung-galaxy-f41-fu...
5,Samsung,Galaxy F62,Laser Grey,6 GB,128 GB,Exynos 9825,Octa Core,7000 mAh,64MP + 12MP + 5MP + 5MP,32MP Front Camera,17.02 cm (6.7 inch),2400 x 1080 Pixels,"₹23,999",https://www.flipkart.com/samsung-galaxy-f62-la...
6,Samsung,Galaxy F62,Laser Blue,6 GB,128 GB,Exynos 9825,Octa Core,7000 mAh,64MP + 12MP + 5MP + 5MP,32MP Front Camera,17.02 cm (6.7 inch),2400 x 1080 Pixels,"₹23,999",https://www.flipkart.com/samsung-galaxy-f62-la...
7,Samsung,Galaxy M51,Electric Blue,8 GB,128 GB,Octa Core,Octa Core,7000 mAh,64MP Rear Camera,-,17.02 cm (6.7 inch),1080 x 2400$$Pixels,"₹25,078",https://www.flipkart.com/samsung-galaxy-m51-el...
8,Samsung,Galaxy A21s,Blue,4 GB,64 GB,Exynos 850 Octa Core,Octa Core,5000 mAh,48MP + 8MP + 2MP + 2MP,13MP Front Camera,16.51 cm (6.5 inch),1600 x 720,"₹13,999",https://www.flipkart.com/samsung-galaxy-a21s-b...
9,Samsung,Galaxy M01,Blue,3 GB,32 GB,Qualcomm Snapdragon (SDM439) Octa Core,Octa Core,4000 mAh,13MP + 2MP,5MP Front Camera,14.48 cm (5.7 inch),1520 x 720 Pixels,"₹7,809",https://www.flipkart.com/samsung-galaxy-m01-bl...


## 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [55]:
def coordinates(url):
    
    Location = input('Enter the location : ')
#     location = input('Enter location name :')
    driver = webdriver.Chrome(r'C:\Users\Rahul\Downloads\chromedriver_win32\chromedriver.exe',chrome_options=options)
    driver.get(url)
    
    #search box
    search_box = driver.find_element_by_xpath("//input[@id='searchboxinput']")
    search_box.send_keys(Location)
    
    #clicking search button
    search_button = driver.find_element_by_xpath("//button[@id='searchbox-searchbutton']")
    search_button.click()
    time.sleep(5) #time is given so that the page will get load completely and we will have the url updated
    
    #now the coordinates details are available at the url of searched page so
    #we will be extracting the coordinates details from url itself.
    
    url_extracted = driver.current_url
#     print(url_extracted)
    
    #updating the current url into a variable
    try:
        url_string = url_extracted
#         print("URL_fetched: ",url_string)
# using regex here to extract the pattern of latitude and longitude
        str1 = re.findall(r'@.*\dz',url_string)
        lat_long1 = str1[0].split(',')[0:2]
        longitude = lat_long1[1]
        print("Longitude : ",longitude)
        
        latitude = lat_long1[0].split('@')[1]
        print("Latitude : ", latitude)
    except NoSuchElementException:
        print('-')

In [56]:
coordinates("https://www.google.com/maps/")

Enter the location : Himachal Pradesh
URL_fetched:  https://www.google.com/maps/place/Himachal+Pradesh/@31.8152979,76.1741208,8z/data=!4m5!3m4!1s0x390453c367f901f7:0x7cfe04c5564b7725!8m2!3d31.1048294!4d77.1733901
Longitude :  76.1741208
Latitude :  31.8152979


In [57]:
string = "https://www.google.com/maps/place/Jaipur,+Rajasthan/@26.8851417,75.6504714,11z/data=!3m1!4b1!4m5!3m4!1s0x396c4adf4c57e281:0xce1c63a0cf22e09!8m2!3d26.9124336!4d75.7872709"

In [58]:
str1 = re.findall(r'@.*\dz',string)

In [59]:
str1

['@26.8851417,75.6504714,11z']

In [64]:
lat_long1 = str1[0].split(',')[0:2]

In [65]:
lat_long1

['@26.8851417', '75.6504714']

In [67]:
lat_long1[0].split('@')[1]

'26.8851417'